In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv("sales_pos.csv")
df.head(2)

user       prod gender age_group  job city  marital  prod_cat1  prod_cat2   
0     1  P00069042      F      0-17   10    A        0          3        NaN  \
1     1  P00248942      F      0-17   10    A        0          1        6.0   

   prod_cat3  purchase  
0        NaN      8370  
1       14.0     15200

### Q1.

In [3]:
df_q1 = df[["prod", "job", "purchase"]].copy()
df_q1.head(2)

prod  job  purchase
0  P00069042   10      8370
1  P00248942   10     15200

In [5]:
# df_q1.groupby("prod")["purchase"].sum()
df_q1.groupby("prod")["purchase"].sum().max()

27995166

In [7]:
top_prod = df_q1.groupby("prod")["purchase"].sum().idxmax()
top_prod

'P00025442'

In [9]:
df_q1_sub = df_q1.loc[df_q1["prod"] == top_prod, ]
df_q1_sub.head(2)

prod  job  purchase
667  P00025442   17     19706
749  P00025442    7     15212

In [11]:
df_q1_sub["job"].value_counts().idxmax()

4

### Q2.

In [12]:
df.head(1)

user       prod gender age_group  job city  marital  prod_cat1  prod_cat2   
0     1  P00069042      F      0-17   10    A        0          3        NaN  \

   prod_cat3  purchase  
0        NaN      8370

In [21]:
df_u1 = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]]
df_u1 = df_u1.fillna(0).reset_index(drop = True)
df_u1.head()

prod_cat1  prod_cat2  prod_cat3
0          3        0.0        0.0
1          1        6.0       14.0
2         12        0.0        0.0
3         12       14.0        0.0
4          2        4.0        8.0

In [15]:
len(df_u1)

35

In [18]:
df_u1.drop_duplicates().shape

(21, 3)

In [22]:
df_u1["prod_cat1"] = df_u1["prod_cat1"].astype("int").astype("str")
df_u1["prod_cat2"] = df_u1["prod_cat2"].astype("int").astype("str")
df_u1["prod_cat3"] = df_u1["prod_cat3"].astype("int").astype("str")
df_u1["prod_cat"] = df_u1["prod_cat1"] + "-" + df_u1["prod_cat2"] + "-" + df_u1["prod_cat3"]
df_u1.head()

prod_cat1 prod_cat2 prod_cat3 prod_cat
0         3         0         0    3-0-0
1         1         6        14   1-6-14
2        12         0         0   12-0-0
3        12        14         0  12-14-0
4         2         4         8    2-4-8

In [23]:
df_u1["prod_cat"].nunique()

21

In [24]:
df["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [27]:
cols = ["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]
df_q2 = df.loc[df["age_group"] == "26-35", cols].reset_index(drop = True)
df_q2 = df_q2.fillna(0)
df_q2.head(2)

user  marital  prod_cat1  prod_cat2  prod_cat3
0     3        0          1        2.0        0.0
1     5        1          8        0.0        0.0

In [28]:
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("int").astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2["prod_cat"]  = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]
df_q2.head()

user  marital prod_cat1 prod_cat2 prod_cat3 prod_cat
0     3        0         1         2         0    1-2-0
1     5        1         8         0         0    8-0-0
2     5        1         5        11         0   5-11-0
3     5        1         8         0         0    8-0-0
4     5        1         8         0         0    8-0-0

In [32]:
df_q2_agg = df_q2.groupby(["user", "marital"])["prod_cat"].nunique().reset_index()
df_q2_agg.head()

user  marital  prod_cat
0     3        0        18
1     5        1        43
2     8        1        32
3     9        0        31
4    11        0        34

In [33]:
stat_m0 = df_q2_agg.loc[df_q2_agg["marital"] == 0, "prod_cat"].mean()
stat_m1 = df_q2_agg.loc[df_q2_agg["marital"] == 1, "prod_cat"].mean()
round(abs(stat_m0 - stat_m1), 2)

0.13

In [35]:
df_q2_agg["value"] = 1
df_q2_agg.head(2)

user  marital  prod_cat  value
0     3        0        18      1
1     5        1        43      1

In [37]:
df_q2_agg.groupby("marital")[["prod_cat", "value"]].agg({"prod_cat": "min",
                                                         "value": "max"})

prod_cat  value
marital                 
0               5      1
1               6      1

### Q3.

In [39]:
df_user = df[["user", "gender", "age_group", "job", "city", "marital"]]
df_user = df_user.drop_duplicates().reset_index(drop = True)
df_user.shape

(5891, 6)

In [40]:
df_user.head(2)

user gender age_group  job city  marital
0     1      F      0-17   10    A        0
1     2      M       55+   16    C        0

In [41]:
df["user"].nunique()

5891

In [43]:
df_agg1 = df.groupby("user")["prod"].nunique().reset_index()
df_agg2 = df.groupby("user")["purchase"].sum().reset_index()
df_agg1.head(2)

user  prod
0     1    35
1     2    77

In [44]:
df_join = df_user.merge(df_agg1, on = "user")
df_join = df_join.merge(df_agg2, on = "user")
df_join.head(2)

user gender age_group  job city  marital  prod  purchase
0     1      F      0-17   10    A        0    35    334093
1     2      M       55+   16    C        0    77    810472

In [45]:
df_join["gender"] = df_join["gender"].replace({"M": 1, "F": 0})

In [47]:
arr_age = df_join["age_group"].unique()
arr_age.sort()

In [48]:
arr_age

array(['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'],
      dtype=object)

In [49]:
ser_repl = pd.Series(range(len(arr_age)), index = arr_age)
ser_repl

0-17     0
18-25    1
26-35    2
36-45    3
46-50    4
51-55    5
55+      6
dtype: int64

In [50]:
pd.crosstab(df_join["age_group"], df_join["age_group"].replace(ser_repl))

age_group    0     1     2     3    4    5    6
age_group                                      
0-17       218     0     0     0    0    0    0
18-25        0  1069     0     0    0    0    0
26-35        0     0  2053     0    0    0    0
36-45        0     0     0  1167    0    0    0
46-50        0     0     0     0  531    0    0
51-55        0     0     0     0    0  481    0
55+          0     0     0     0    0    0  372

In [51]:
df_join["age_group"] = df_join["age_group"].replace(ser_repl)

In [52]:
df_join.head(2)

user  gender  age_group  job city  marital  prod  purchase
0     1       0          0   10    A        0    35    334093
1     2       1          6   16    C        0    77    810472

In [55]:
# df_join_dum = pd.get_dummies(df_join, columns = ["job", "city"]) # 시험버전
df_join_dum = pd.get_dummies(df_join, columns = ["job", "city"],
                             dtype = "int")
df_join_dum = df_join_dum.drop(columns = "user")
df_join_dum.head(2)

gender  age_group  marital  prod  purchase  job_0  job_1  job_2  job_3   
0       0          0        0    35    334093      0      0      0      0  \
1       1          6        0    77    810472      0      0      0      0   

   job_4  ...  job_14  job_15  job_16  job_17  job_18  job_19  job_20  city_A   
0      0  ...       0       0       0       0       0       0       0       1  \
1      0  ...       0       0       1       0       0       0       0       0   

   city_B  city_C  
0       0       0  
1       0       1  

[2 rows x 29 columns]

In [56]:
arr_join_nor = MinMaxScaler().fit_transform(df_join_dum)
df_join_nor = pd.DataFrame(arr_join_nor, columns = df_join_dum.columns)
df_join_nor.head(2)

gender  age_group  marital      prod  purchase  job_0  job_1  job_2  job_3   
0     0.0        0.0      0.0  0.028431  0.027398    0.0    0.0    0.0    0.0  \
1     1.0        1.0      0.0  0.069608  0.072810    0.0    0.0    0.0    0.0   

   job_4  ...  job_14  job_15  job_16  job_17  job_18  job_19  job_20  city_A   
0    0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0     1.0  \
1    0.0  ...     0.0     0.0     1.0     0.0     0.0     0.0     0.0     0.0   

   city_B  city_C  
0     0.0     0.0  
1     0.0     1.0  

[2 rows x 29 columns]

In [ ]:
model_km = KMeans(n_clusters = 7, random_state = 123)
model_km.fit(df_join_nor)

In [60]:
# model_km.cluster_centers_
# model_km.labels_ # <-- 각 row에 할당되는 군집 번호

In [61]:
round(silhouette_score(df_join_nor, labels = model_km.labels_), 2)

0.18

In [63]:
pd.Series(model_km.labels_).value_counts() 
# 가장 크기가 큰 군집은 1번 군집이다.

1    1291
0    1004
5     980
2     868
6     703
3     652
4     393
Name: count, dtype: int64